Parsing and head automata
----

Decoding parse trees with head automata for consecutive siblings

In [1]:
import numpy as np 
import ad3
from ad3.extensions import PFactorTree, PFactorHeadAutomaton

Initialize variables

In [2]:
g = ad3.factor_graph.PFactorGraph()
arc_variables = []
arcs = []

num_tokens = 5  # including root

Set up scores

In [3]:
# create indices and scores from every node to all others
scores = np.random.normal(0, 1, [num_tokens, num_tokens])

# root is never a modifier
scores[:, 0] -= 1000

scores[0, 2] += 5
scores[2, 1:] += 5

Add the tree factor to the graph

In [4]:
index_arcs = np.full([num_tokens, num_tokens], -1, np.int)

# suppose the arc 0 -> 1 has been pruned
filtered_arcs = set([(0, 1)])

for h in range(num_tokens):
    for m in range(1, num_tokens):
        if h == m or (h, m) in filtered_arcs:
            continue
            
        arcs.append((h, m))
        arc_var = g.create_binary_variable()
        arc_var.set_log_potential(scores[h, m])
        index_arcs[h, m] = len(arc_variables)
        arc_variables.append(arc_var)

tree = PFactorTree()
g.declare_factor(tree, arc_variables)
tree.initialize(num_tokens, arcs)

In [5]:
for arc in arcs:
    h, m = arc
    score = scores[h, m]
    print(arc, score)

(0, 2) 4.827298240886875
(0, 3) 1.5034222682618876
(0, 4) 1.0873765691778632
(1, 2) -0.16082610907657346
(1, 3) 0.710943513720957
(1, 4) 1.1240017780850429
(2, 1) 4.99152398174798
(2, 3) 4.558038158837646
(2, 4) 5.985031077477877
(3, 1) 0.26904093592443573
(3, 2) -0.6235655113919879
(3, 4) -0.15193711113079478
(4, 1) 1.5105968086408372
(4, 2) 0.5263399549513429
(4, 3) -0.14857711201037666


Solve for the exact solution

In [6]:
value, posteriors, additional_posteriors, status = g.solve_exact_map_ad3()
print(value)
print(posteriors)
print(additional_posteriors)
print(status)

20.361891458950378
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[]
0


In [7]:
def show_selected_items(posteriors, arcs, show_posterior=False):
    for p, arc in zip(posteriors, arcs):
        if p ** 2 <= 0.0001:
            continue
        if show_posterior:
            print(arc, p)
        else:
            print(arc)


In [8]:
show_selected_items(posteriors, arcs)

(0, 2)
(2, 1)
(2, 3)
(2, 4)


Now, let's add next sibling features and head automata

In [9]:
def create_empty_lists(n):
    """
    Create a list with n empty lists
    """
    return [[] for _ in range(n)]


# create all possible sibling combinations considering non-pruned arcs
sibling_parts_left = create_empty_lists(num_tokens)
sibling_parts_right = create_empty_lists(num_tokens)

for h in range(num_tokens):
    # right side
    for m in range(h, num_tokens):
        if h != m and index_arcs[h, m] == -1:
            continue
            
        for s in range(m + 1, num_tokens + 1):
            # s == num_tokens signals that m is the rightmost child
            if s < num_tokens and index_arcs[h, s] == -1:
                continue
            
            sibling_part = (h, m, s)
            sibling_parts_right[h].append(sibling_part)
    
    # left side
    if h == 0:
        # root has no children to the left
        continue
        
    for m in range(h, 0, -1):        
        if h != m and (h, m) in filtered_arcs:
            continue
        
        for s in range(m - 1, -1, -1): 
            # s == 0 signals that m is the leftmost child
            if s > 0 and (h, s) in filtered_arcs:
                continue
            
            sibling_part = (h, m, s)
            sibling_parts_left[h].append(sibling_part)

# create random scores for each sibling part
# (we can't create an array here because each head has a different number of sibling parts)
sibling_scores_left = [np.random.normal(size=len(siblings)) 
                       for siblings in sibling_parts_left]
sibling_scores_right = [np.random.normal(size=len(siblings)) 
                        for siblings in sibling_parts_right]

Include sibling factors in the graph

In [10]:
factors = []
additionals = []

# there is a factor for each head and side combination
for h in range(num_tokens):
    
    # right hand size
    
    # local_vars indicates which variables are constrained by the factor
    local_vars = []
    local_siblings = []
    additional_scores = []
    local_arcs = []
    
    for m in range(h + 1, num_tokens):
        index = index_arcs[h, m]
        if index < 0:
            continue

        local_vars.append(arc_variables[index])
        local_arcs.append((h, m))
        
    factor = PFactorHeadAutomaton()
    factors.append(factor)
    
    # important: first declare the factor in the graph, then initialize
    # it may seems counter-intuitive but breaks otherwise
    g.declare_factor(factor, local_vars)
    length = num_tokens - h if h > 0 else num_tokens - 1
    factor.initialize(local_arcs, sibling_parts_right[h], validate=False)
    factor.set_additional_log_potentials(sibling_scores_right[h])
    additionals.extend(sibling_parts_right[h])
    
    # left hand size
    if h == 0:
        # root has no children to the left
        continue
        
    local_vars = []
    local_siblings = []
    additional_scores = []
    local_arcs = []
    for m in range(h, 0, -1):
        index = index_arcs[h, m]
        if index < 0:
            continue
            
        local_vars.append(arc_variables[index])
        local_arcs.append((h, m))
        
    factor = PFactorHeadAutomaton()
    factors.append(factor)
    
    # important: first declare the factor in the graph, then initialize
    # it may seems counter-intuitive but breaks otherwise
    g.declare_factor(factor, local_vars)
    factor.initialize(local_arcs, sibling_parts_left[h], validate=False)
    factor.set_additional_log_potentials(sibling_scores_right[h])
    additionals.extend(sibling_parts_left[h])

Solve again, now with the siblings

In [11]:
value, posteriors, additional_posteriors, status = g.solve_exact_map_ad3()
print(value)
print(posteriors)
print(additional_posteriors)
print(status)

3.739708487905299
[2.220446049250313e-16, 0.0, 0.9999999999999991, 0.0, 0.0, 0.0, 0.0, 0.9999999999999997, 0.0, 0.0, 0.0, 0.0, 0.9999999999999997, 0.9999999999999997, 0.0]
[0.0, 0.0, 0.9999999999999989, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999989, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
0


In [12]:
show_selected_items(posteriors, arcs)

(0, 4)
(2, 3)
(4, 1)
(4, 2)


In [13]:
show_selected_items(additional_posteriors, additionals)

(0, 0, 4)
(0, 4, 5)
(1, 1, 5)
(1, 1, 0)
(3, 3, 4)
(3, 3, 1)
(3, 2, 0)
(4, 4, 0)
(4, 2, 1)
(4, 2, 0)
